# ML Trader

In [1]:
from utils.packages import *
from utils.variables import *
from utils.i_o import *
from utils.dir_slope import *
from utils.engine import *
from utils.loops import *
from utils.order import *
from utils.ml import *
import winsound

# Notes
---
## 2022_09_02

### Observations

1. No need to make 10 transactions a day. Just one is enough

1. lema or sema match
    1. Using lema match works fine in both opening 
    1. Using lema has better results
    1. 144 produces better resuls over higher numbers like 1440 

1. Lema angle in order and close
    1. Using lema angle in order, delays order, but overall profits are high
    1. Using lema angle in close also increases overall profit

1. Before everything need to find why 2021 is consistently producing negative results in all the setups.
    1. What is that this causing this issue, how can it be addressed with in the same system
    1. less complex method is essential
    

### To Test
1. Find whether the opening or closing that is causing the most issue in loss years
1. Test other years to understand how well this setup works.
1. Remove lema match check from closing. This is delaying bad trasactions
1. Test 5 round up on lema match. Transactions are opening in lema match gaps
1. Ideation
    1. Test lema Bollinger bands to understand flow direction
    1. Test overall flow direction

---

## 2022_09_02

### Observations
1. Opening and closing need to happen faster. Cant close after ema match
1. Lema angle helps save positions against frequent flips but delays closing and causes losses
1. Market is non-directional (Dec 2021 is a good example) Goes up and down, when algo tries to open and close its always too late
1. Need a faster method


### To Test
1. Should test plotting close as a line instead of candles. Perform match in close with wider window
1. Close chart needs to be analysed. This looks promising.
1. Go with simple rolling min max for close. Works better than resample min max merge

---

## Result by Years

| Year | Angle open | Angle close | Lema match open | Lema match close | Result type | pips  |
| ---- | ---------- | ----------- | --------------- | ---------------- | ----------- | ----- |
| 2022 | Yes        | No          | Yes             | Yes              | Profit      | 200   |
| 2021 | Yes        | No          | Yes             | Yes              | Loss        | \-800 |
| 2020 | Yes        | No          | Yes             | Yes              | Profit      | 800   |
| 2019 | Yes        | No          | Yes             | Yes              | Loss        | \-400 |
| 2018 | Yes        | No          | Yes             | Yes              | Profit      | 300   |
| 2017 | Yes        | No          | Yes             | Yes              | Profit      | 14    |    

data                = get_date_list(data)
data                = read_data(data)

In [2]:
%%time
data = run_engine(data)

100%|██████████████████████████████████▉| 74226/74266 [02:59<00:00, 340.67it/s]

pl   -0.6936
dtype: float64
--------------------------------------
            DateTime_frmt  order_side  close_type       pl
74122 2021-12-31 10:00:00         NaN  sema_close -0.00053
74123 2021-12-31 10:05:00         NaN  sema_close -0.00038
74124 2021-12-31 10:10:00         NaN  sema_close -0.00035
74128 2021-12-31 10:30:00         NaN  sema_close -0.00033
74135 2021-12-31 11:05:00         NaN  sema_close  0.00029
74139 2021-12-31 11:25:00         NaN  sema_close  0.00027
74140 2021-12-31 11:30:00         NaN  sema_close -0.00044
74141 2021-12-31 11:35:00         NaN  sema_close -0.00052
74149 2021-12-31 12:15:00         NaN  sema_close  0.00063
74156 2021-12-31 12:50:00         NaN  sema_close  0.00018
74163 2021-12-31 13:25:00         NaN  sema_close -0.00021
74172 2021-12-31 14:10:00         NaN  sema_close  0.00117
74191 2021-12-31 15:45:00         NaN  sema_close  0.00021
74201 2021-12-31 16:35:00         NaN  sema_close -0.00030
74213 2021-12-31 17:35:00         NaN  sema_clos

100%|█████████████████████████████████████| 7319/7319 [00:20<00:00, 361.33it/s]

Wall time: 4min 57s


In [3]:
x = data['df_ohlc'].groupby(['month_val'])['pl'].sum()
print(x)
print('---------------------')
print(x.sort_values())
print('---------------------')
print(f'max : {x.idxmax()} : {max(x)}')
print(f'min : {x.idxmin()} : {min(x)}')
print('---------------------')
monthly_avg = np.round(data['df_ohlc']['pl'].sum()/len(set(data['df_ohlc']['month_val'])),4)
print(f'monthly_avg : {monthly_avg}')

month_val
1    -0.07831
2    -0.05484
3    -0.08175
4    -0.05788
5    -0.04691
6    -0.05989
7    -0.06462
8    -0.05032
9    -0.07193
10   -0.02492
11   -0.05985
12   -0.04238
Name: pl, dtype: float64
---------------------
month_val
3    -0.08175
1    -0.07831
9    -0.07193
7    -0.06462
6    -0.05989
11   -0.05985
4    -0.05788
2    -0.05484
8    -0.05032
5    -0.04691
12   -0.04238
10   -0.02492
Name: pl, dtype: float64
---------------------
max : 10 : -0.02492000000000001
min : 3 : -0.08174999999999989
---------------------
monthly_avg : -0.0578


In [4]:
sys.exit()

SystemExit: 

In [ ]:
print(data['df_small'].groupby(['weekday', 'pl_type'])['pl_type'].count().unstack())
print('----------------------------------------------')
print(data['df_small'].groupby(['weekday', 'pl_type'])['pl'].sum().unstack())

In [ ]:
data['lema_gap_pl_analysis_month'] = 3

data = lema_gap_pl_analysis(data)
data['lema_gap_pl_report']

In [ ]:
y = data['df_ohlc']['pl'].sum()
x = data['df_ohlc'].groupby(['month_val'])['pl'].sum()
send_telegram_message(f'{data["input_year"]}')
send_telegram_message(f'{y}')
send_telegram_message(f'{x}')
send_telegram_message(f'-------------------')

In [ ]:
y = data['df_ohlc'][data['df_ohlc']['month_val'] == 1]
x = y.groupby(['date_val'])['pl'].sum()
print(x)
print('---------------------')
print(f'max : {x.idxmax()} : {max(x)}')
print(f'min : {x.idxmin()} : {min(x)}')
print('---------------------')

In [ ]:
data['df_ohlc'].tail()

In [ ]:
data['df_ohlc']['lema_gap'].describe()

In [ ]:
x = 5
data['df_ohlc'].iloc[-5:,x:x+15]

In [ ]:
data['df_small']['lema_gap'].describe()

In [ ]:
a = np.arange(0,20)
for b in a:
    x = int(round(b/5.0)*5.0)
    print(b, x)

In [ ]:
abs(all['negative_sum']) / all['positive_sum']

In [ ]:
df_small['order_side']

In [ ]:
data['df_small'].to_csv('data/temp.csv')

In [5]:
data["plot"] = True
data['plot_type'] = 'file'
data['reduce_plot'] = True
chart_duration = 'month'
month = 1

# data['things_to_plot'] = ['tick', 'ema', 'BBands', 'dots']
# data['things_to_plot'] = ['ema', 'BBands', 'dots', 'indicators']
# data['things_to_plot'] = ['ema', 'BBands', 'dots', 'cross']
# data['things_to_plot'] = ['ema', 'BBands', 'dots', 'positions', 'tip']
# data['things_to_plot'] = ['ema', 'BBands', 'dots', 'positions', 'lema_angle']
# data['things_to_plot'] = ['ema', 'sema_angle', 'slema_angle', 'lema_angle', 'dir']
# data['things_to_plot'] = ['ema', 'positions', 'lema_angle','slema_angle','sema_angle', 'rsi']
# data['things_to_plot'] = ['ema','positions', 'lema_angle','lema_diff', 'lema_max_min']
# data['things_to_plot'] = ['candles','sema', 'slema','positions','lema', 'lema_angle', 'lema_max_min', 'close_diff']
# data['things_to_plot'] = ['lema', 'tip', 'candles', 'lema_angle', 'lema_change']
# data['things_to_plot'] = ['candles','sema', 'close','lema', 'lema_match', 'positions', 'lema_change']
# data['things_to_plot'] = ['close', 'sema','lema', 'lema_change', 'lema_match', 'close_max']
# data['things_to_plot'] = ['close', 'tip','lema', 'close_max', 'close_min', 'lema_match']
data['things_to_plot'] = ['close', 'sema','lema_match' ,'positions']
# data['things_to_plot'] = ['close']

data['plot_start']  = f"{data['input_year']}-{month}-1"

if chart_duration == 'month':
    if month == 12:
        data['plot_stop']   = f"{data['input_year']+1}-1-1"
    else:
        data['plot_stop']   = f"{data['input_year']}-{month+1}-1"

elif chart_duration == 'year':        
    data['plot_start']  = f"{data['input_year']}-1-1"
    data['plot_stop']   = f"{data['input_year']+1}-1-1"


plot_graph(data)

generate_result_report(data)

In [ ]:
data['df_ohlc']['close_max'] = data['df_ohlc']['close'].rolling(window = data['close_min_max_duration']).max()
data['df_ohlc']['close_min'] = data['df_ohlc']['close'].rolling(window = data['close_min_max_duration']).min()

In [ ]:
data['df_ohlc']['close_max'][0:10]

In [ ]:
data['df_ohlc']['close_max'][0:10]

In [ ]:
data['df_ohlc']['close_max'].duplicated()[0:10]

In [ ]:
cols = ['DateTime_frmt', 'close', 'ask','bid', 
 'open', 'high', 'low', 'candle_size', 'close_max', 'close_min', 'lema', 'lema_angle', 'lema_angle_2',
       'lema_angle_0', 'slema', 'slema_angle', 'slema_angle_2', 'sema',
       'sema_angle', 'sema_angle_2', 'close_diff', 'BBand_upper',
       'BBand_middle', 'BBand_lower', 'BBand_width', 'avg_BBand_width', 'adx',
       'tip', 'lema_match', 'order_side', 'order_size', 'long_open',
       'short_open', 'close_type', 'pl', 'up', 'down', 'month_val', 'date_val',
       'hour_val', 'long_profit', 'long_loss', 'short_profit', 'short_loss']

data['df_ohlc'][cols].to_csv('data/temp.csv')

In [ ]:
data['df_ohlc'].loc[~(data['df_ohlc']['close'] == data['df_ohlc']['close_max']), 'close_max'] = np.nan
data['df_ohlc'].loc[~(data['df_ohlc']['close'] == data['df_ohlc']['close_min']), 'close_min'] = np.nan

In [ ]:
data['df_ohlc'][['close', 'close_max', 'close_min']][0:20]

In [ ]:
data['df_ohlc'].loc[data['df_ohlc']['close_max'].duplicated(), 'close_max'] = np.nan
data['df_ohlc'].loc[data['df_ohlc']['close_min'].duplicated(), 'close_min'] = np.nan